# Fixed effects and first differencing versus diffs in diffs
Have always been a little confused as to whether to use diffs in diffs or fixed effects when we have panel data with two time periods.  In this notebook, I generate some fake data to simulate the effect of using one versus the other.

In [10]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [58]:
% matplotlib inline
sampsize = 1000
pre = np.random.normal(0,1, sampsize)
treat = np.random.binomial(1,.5,sampsize)
# add a little bit of extra noise just to the "treated" units
pre = pre + treat*np.random.normal(.1,.05,sampsize)
noise = np.random.normal(0,.15,sampsize)
effect = .1
post = pre + noise + effect*treat

In [59]:
# estimate the effect using diffs in diffs
outcome = np.concatenate((pre,post),0)
repeat_treat = np.concatenate((treat,treat),0)
pre_post = np.concatenate((np.repeat(0,sampsize),np.repeat(1,sampsize)),0)
df = pd.DataFrame({'y':outcome, 'treat':repeat_treat,'round':pre_post})

In [60]:
# estimate the unadjusted impact on wealthindex_e
import statsmodels.api as sm
X = np.stack((pre_post, repeat_treat, pre_post*repeat_treat),1)
X = sm.add_constant(X)
model = sm.OLS(df.y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.017
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     11.69
Date:                Mon, 25 Jul 2016   Prob (F-statistic):           1.36e-07
Time:                        20:34:06   Log-Likelihood:                -2798.6
No. Observations:                2000   AIC:                             5605.
Df Residuals:                    1996   BIC:                             5628.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const         -0.0050      0.044     -0.113      0.9

In [61]:
# Estimate the same model using a first difference approach
delta_y = post - pre
delta_treat = treat
X = delta_treat
X = sm.add_constant(X)
model = sm.OLS(delta_y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.111
Model:                            OLS   Adj. R-squared:                  0.110
Method:                 Least Squares   F-statistic:                     124.2
Date:                Mon, 25 Jul 2016   Prob (F-statistic):           2.92e-27
Time:                        20:34:07   Log-Likelihood:                 480.31
No. Observations:                1000   AIC:                            -956.6
Df Residuals:                     998   BIC:                            -946.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const         -0.0131      0.007     -1.944      0.0

In [65]:
# one more time using fixed effects

array([ 0.00666126,  0.0094819 ])

# Key Results
For the very simple case with no covariates presented above, the estimates of the impact are always the same but...
1. The estimates of impact are always the same.
2. When there is no difference in treat and control in the pre period (as you would expect in an RCT), the estimate of the standard errors of the estimators are pretty similar
3. If there is a systematic difference between treat and control, the standard error of the first difference estimator can be much smaller than the standard error of the diffs in diffs estimator

A couple of other things to consider

1. With fixed effects / first differencing, you can't include covariates that don't change over time

# More stuff to do
1. Ideally, I should derive the estimates of variance for the different estimators and see how they differ
2. 
